In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import sys
import time
import numpy as np
from sklearn.metrics import f1_score
import re

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

'''
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''
clinical_path_train = "/kaggle/input/olives-training-labels/Training_Unlabeled_Clinical_Data.csv"
clinical_path_test = "/kaggle/input/olives-training-labels/test_with_clinical.csv"
min_cst, max_cst = 150, 723

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# model.py

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision

class ResNet(nn.Module):
    """encoder + classifier"""
    def __init__(self, name='resnet50', num_classes=2):
        super(ResNet, self).__init__()
        if (name == 'resnet50'):
            self.encoder = torchvision.models.resnet50(zero_init_residual=True)
            self.encoder.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
            self.encoder.fc = nn.Identity()
            self.fc = nn.Linear(2048, num_classes)
        else:
            self.encoder = torchvision.models.resnet18(zero_init_residual=True)
            self.encoder.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
            self.encoder.fc = nn.Identity()
            self.fc = nn.Linear(512, num_classes)
    def forward(self, x):

        return self.fc(self.encoder(x))



In [3]:
class ResNetModified(nn.Module):
    """encoder + classifier"""
    def __init__(self, name='resnet50', num_classes=2):
        super(ResNetModified, self).__init__()
        if (name == 'resnet50'):
            self.encoder = torchvision.models.resnet50(zero_init_residual=True)
            self.encoder.conv1 = nn.Conv2d(1, 64, kernel_size=(9, 9), stride=(2, 2), padding=(4, 4), bias=False)
            self.encoder.fc = nn.Identity()
            self.fc = nn.Linear(2048, num_classes)
        else:
            self.encoder = torchvision.models.resnet18(zero_init_residual=True)
            self.encoder.conv1 = nn.Conv2d(1, 64, kernel_size=(9, 9), stride=(2, 2), padding=(4, 4), bias=False)
            self.encoder.fc = nn.Identity()
            self.fc = nn.Linear(512, num_classes)
    def forward(self, x):

        return self.fc(self.encoder(x))

In [4]:
# For clinical labels
class MLP(nn.Module):
    # define model elements
    def __init__(self, n_inputs, n_outputs):
        super(MLP, self).__init__()
        self.layer = nn.Sequential(

            # for clinical label only baseline
            nn.Linear(n_inputs, n_inputs),
            nn.LeakyReLU(0.2, True),
            nn.Linear(n_inputs, n_outputs)
        )

    # forward propagate input
    def forward(self, X):
        X = self.layer(X)
        return X

In [5]:
import torch
import torch.nn as nn

class HybridModel(nn.Module):
    def __init__(self, name, num_classes, num_clinical):
        super(HybridModel, self).__init__()

        if (name == 'resnet50'):
            self.encoder = torchvision.models.resnet50(zero_init_residual=True, pretrained = True)
            self.encoder.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
            self.encoder.fc = nn.Identity()
            self.encoder_fc = nn.Linear(2048, 10)
        else:
            self.encoder = torchvision.models.resnet18(zero_init_residual=True)
            self.encoder.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
            self.encoder.fc = nn.Identity()
            self.encoder_fc = nn.Linear(512, 10)
        
        # Combine the outputs of CNN and numeric layers
        combined_features_dim = 10 + num_clinical
        self.fc = nn.Sequential(
            nn.LeakyReLU(0.2, True),
            nn.Linear(combined_features_dim, num_classes)
        )

    def forward(self, image_data, numeric_data):
        # Process image data using CNN layers
        image_features = self.encoder_fc(self.encoder(image_data))

        # Concatenate image and numeric features
        combined_features = torch.cat((image_features.view(image_features.size(0), -1), numeric_data), dim=1)

        # Final prediction using combined features
        output = self.fc(combined_features)
        return output


In [6]:
class Encoder(nn.Module):
    """encoder + classifier"""
    def __init__(self, name='resnet50', num_classes=2):
        super(Encoder, self).__init__()
        self.encoder = torchvision.models.resnet50(pretrained=True, zero_init_residual=True)
        self.encoder.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.encoder.fc = nn.Identity()
        self.fc = nn.Linear(2048, 512)

    def forward(self, x):

        return self.fc(self.encoder(x))


class ProjectionHead(nn.Module):
    def __init__(self,in_dim,feature_dim):
        super(ProjectionHead, self).__init__()
        
        self.g1 = nn.Sequential(nn.Linear(in_dim, 1024, bias=False),
                               nn.BatchNorm1d(1024),
                               nn.ReLU(inplace=True)
                               )
        self.g2 = nn.Sequential(nn.Linear(1024, 512, bias=False),
                                nn.BatchNorm1d(512),
                                nn.ReLU(inplace=True)
                                )
        self.g3=nn.Linear(512, feature_dim, bias=True)
    def forward(self, x):
        # print(x.shape)
        x = torch.flatten(x, start_dim=1, end_dim=- 1) 
        x = self.g1(x)
        x = self.g2(x)
        x = self.g3(x)
        return x

In [7]:
def prepare_clinical_data_train(bio_data_path, clinical_data_path):
    
    df = pd.read_csv(bio_data_path)
    clinical_df = pd.read_excel(clinical_data_path)
    filtered_df = pd.DataFrame(columns = clinical_df.columns)
    file_names = list(df.iloc[:, 0])

    for i in range(len(clinical_df)):
        if clinical_df.iloc[i].File_Path in file_names:
            filtered_df.loc[len(filtered_df.index)] = clinical_df.loc[i]
    print("Created clinical data frame")

    new_df = df.copy()
    file_names = list(filtered_df.iloc[:, 0])
    for i in range(len(df)):
        if df.iloc[i,0] not in file_names:
            new_df = new_df.drop(i)
    print("Filtered unlabelled data")

    assert len(new_df) == len(filtered_df), (len(new_df), len(filtered_df))

    filtered_df = filtered_df.sort_values(by = [filtered_df.columns[0]])
    new_df = new_df.sort_values(by = [new_df.columns[0]])
    assert new_df.iloc[0, 0] == filtered_df.iloc[0, 0], (new_df.iloc[0, 0], filtered_df.iloc[0, 0])
    
    bcva_mean = np.mean(filtered_df.BCVA)
    bcva_std = np.std(filtered_df.BCVA)
    cst_mean = np.mean(filtered_df.CST)
    cst_std = np.std(filtered_df.CST)
    filtered_df.BCVA = (filtered_df.BCVA - bcva_mean) / bcva_std
    filtered_df.CST = (filtered_df.CST - cst_mean) / cst_std
    
    return new_df, filtered_df

In [8]:
def prepare_clinical_data_test(bio_data_path, clinical_data_path):
    
    df = pd.read_csv(bio_data_path)
    clinical_df = pd.read_excel(clinical_data_path)
    regex = re.compile(r"RECOVERY/OCT/([0-9\-]+)/W([0-9]+)")
    
    cols = ["Patient ID", "BCVA", "CST"]
    id_to_row = dict(zip(clinical_df['Patient ID'], clinical_df.index))
    for col in cols:
        df[col] = np.nan
        
    clinical_df.iloc[28, [3, 4]] = clinical_df.iloc[28, [1, 2]]

    for i in range(len(df)):
        patient_id, week = regex.findall(df.iloc[i, 0])[0]
        week = int(week)
        idx = id_to_row[patient_id]
        df.loc[i, cols[0]] = clinical_df.iloc[idx, 0]
        if week < 50:   # Use test result that is closest to the week of the OCT
            df.loc[i, cols[1]] = clinical_df.loc[idx, "Week 0 BCVA"]
            df.loc[i, cols[2]] = clinical_df.loc[idx, "Week 0 CST"]
        else:
            df.loc[i, cols[1]] = clinical_df.loc[idx, "Final Week BCVA"]
            df.loc[i, cols[2]] = clinical_df.loc[idx, "Final Week CST"]

    return df


bcva_mean = np.mean(df.BCVA)
bcva_std = np.std(df.BCVA)
cst_mean = np.mean(df.CST)
cst_std = np.std(df.CST)
df.BCVA = (df.BCVA - bcva_mean) / bcva_std
df.CST = (df.CST - cst_mean) / cst_std

In [33]:
def scale_labels(df):
    df.BCVA = df.BCVA / 100
    df.CST = (df.CST - min_cst) / (max_cst - min_cst)

In [32]:
# datasets.py

import torch.utils.data as data
from PIL import Image
import numpy as np
import pandas as pd
import os

class OLIVES(data.Dataset):
    def __init__(self,df, img_dir, transforms):
        self.img_dir = img_dir
        self.transforms = transforms
        self.df = pd.read_csv(df)
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        path = self.img_dir + self.df.iloc[idx,0]
        image = Image.open(path).convert("L")
        image = np.array(image)
        image = Image.fromarray(image)
        image = self.transforms(image)
        b1 = self.df.iloc[idx,1]
        b2 = self.df.iloc[idx,2]
        b3 = self.df.iloc[idx,3]
        b4 = self.df.iloc[idx, 4]
        b5 = self.df.iloc[idx, 5]
        b6 = self.df.iloc[idx, 6]
        bio_tensor = torch.tensor([b1, b2, b3, b4, b5, b6])
        return image, bio_tensor
    
class OLIVES_CLINICAL(data.Dataset):
    def __init__(self, bio_df, clinical_df, img_dir, transforms):
        self.img_dir = img_dir
        self.transforms = transforms
        self.df, self.clinical_df = prepare_clinical_data_train(bio_df, clinical_df)
        
    def __len__(self):
        return len(self.clinical_df)

    def __getitem__(self, idx):
        path = self.img_dir + self.df.iloc[idx,0]
        image = Image.open(path).convert("L")
        image = np.array(image)
        image = Image.fromarray(image)
        image = self.transforms(image)
        b1 = self.df.iloc[idx,1]
        b2 = self.df.iloc[idx,2]
        b3 = self.df.iloc[idx,3]
        b4 = self.df.iloc[idx, 4]
        b5 = self.df.iloc[idx, 5]
        b6 = self.df.iloc[idx, 6]
        bio_tensor = torch.tensor([b1, b2, b3, b4, b5, b6])
        
        bcva = self.clinical_df.iloc[idx, 1]
        cst = self.clinical_df.iloc[idx, 2]
        clinical = torch.tensor([bcva, cst])
        
        return image, clinical, bio_tensor




class RECOVERY(data.Dataset):
    def __init__(self,df, img_dir, transforms):
        self.img_dir = img_dir
        self.transforms = transforms
        self.df = pd.read_csv(df)
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        path = self.img_dir + self.df.iloc[idx,0]
        image = Image.open(path).convert("L")
        image = np.array(image)
        image = Image.fromarray(image)
        image = self.transforms(image)
        return image
    
class RECOVERY_CLINICAL(data.Dataset):
    def __init__(self,df, clinical_df, img_dir, transforms):
        self.img_dir = img_dir
        self.transforms = transforms
        self.df = prepare_clinical_data_test(df, clinical_df)
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        path = self.img_dir + self.df.iloc[idx,0]
        image = Image.open(path).convert("L")
        image = np.array(image)
        image = Image.fromarray(image)
        image = self.transforms(image)
        
        bcva = self.df.loc[idx, "BCVA"]
        cst = self.df.loc[idx, "CST"]
        clinical = torch.tensor([bcva, cst])
        
        return image, clinical

class CLINICAL(data.Dataset):
    def __init__(self, df, img_dir, transforms):
        self.img_dir = img_dir
        self.transforms = transforms
        self.df = pd.read_csv(df)
        scale_labels(self.df)
        if len(self.df) > 1000:
            self.df = self.df.iloc[:1000]
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        path = self.img_dir + self.df.iloc[idx,0]
        image = Image.open(path).convert("L")
        image = np.array(image)
        image = Image.fromarray(image)
        image = self.transforms(image)
        
        bcva = self.df.loc[idx, "BCVA"]
        cst = self.df.loc[idx, "CST"]
        clinical = torch.tensor([bcva, cst])
        
        return image, clinical

class RECOVERY_TEST(data.Dataset):
    def __init__(self,df, img_dir, transforms):
        self.img_dir = img_dir
        self.transforms = transforms
        self.df = pd.read_csv(df)
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        path = self.img_dir + self.df.iloc[idx,0]
        image = Image.open(path).convert("L")
        image = np.array(image)
        image = Image.fromarray(image)
        image = self.transforms(image)
        b1 = self.df.iloc[idx,1]
        b2 = self.df.iloc[idx,2]
        b3 = self.df.iloc[idx,3]
        b4 = self.df.iloc[idx, 4]
        b5 = self.df.iloc[idx, 5]
        b6 = self.df.iloc[idx, 6]
        bio_tensor = torch.tensor([b1, b2, b3, b4, b5, b6])
        return image, bio_tensor


In [11]:
# data_preprocessing.py

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import glob
from tqdm import tqdm
from PIL import Image

def combine_excel(csv_dir):
    filenames = glob.glob(csv_dir + "/*.xlsx")
    outputxlsx = pd.DataFrame()

    for file in filenames:
        df = pd.concat(pd.read_excel(file, sheet_name=None), ignore_index=True, sort=False)
        outputxlsx = outputxlsx.append(df, ignore_index=True)

    outputxlsx.to_csv('test_set_labels.csv',index=False)

def analyze_dataframe(csv_dir):
    pass

def process_images(csv_dir):
    df = pd.read_csv(csv_dir)

    for i in tqdm(range(0,len(df))):
        path = df.iloc[i,0]
        im = Image.open(path).convert('L')


def numpy_submission(sub_dir,np_dir):
    np_file  = np.load(np_dir)
    print(len(np_file))
    sub_dir = pd.read_csv(sub_dir)
    print(len(sub_dir))
    for i in range(0,len(sub_dir)):
        sub_dir.iloc[i,1] = np_file[i,0]
        sub_dir.iloc[i, 2] = np_file[i, 1]
        sub_dir.iloc[i, 3] = np_file[i, 2]
        sub_dir.iloc[i, 4] = np_file[i, 3]
        sub_dir.iloc[i, 5] = np_file[i, 4]
        sub_dir.iloc[i, 6] = np_file[i, 5]
    print(sub_dir.head())
    sub_dir.to_csv('baseline_result.csv',index=False)



    #process_images(csv_dir)

In [12]:
def load_model(model, load_file):
    print('==> Loading...')
    checkpoint = torch.load(load_file)
    model.load_state_dict(checkpoint['model'])
    return model

load_file = '/kaggle/input/models/diff_kernel.pth'  # Path to the saved model file

In [13]:
from __future__ import print_function

import math
import numpy as np
import torch.optim as optim
import os
from sklearn.metrics import f1_score
import torch.backends.cudnn as cudnn
from torchvision import transforms, datasets
from torch.utils.data import random_split

import torch.nn as nn
def set_model(opt):


    device = opt.device
    model = ResNet(name=opt.model,num_classes = opt.ncls)
    criterion = torch.nn.BCEWithLogitsLoss()

    model = model.to(device)
    criterion = criterion.to(device)


    return model, criterion

def set_model_hybrid(opt):

    device = opt.device
    model = HybridModel(name=opt.model,num_classes = opt.ncls, num_clinical = 2)
    criterion = torch.nn.BCEWithLogitsLoss()

    model = model.to(device)
    criterion = criterion.to(device)


    return model, criterion

def set_model_hybrid_pretrained(opt, encoder_type = ResNet):

    device = opt.device
    pre_model = encoder_type(name=opt.model,num_classes = opt.ncls)
    criterion = torch.nn.BCEWithLogitsLoss()
    
    pre_model = load_model(pre_model, load_file)
    model = HybridPretrainedModel(pre_model.encoder, opt.model, opt.ncls, 2)

    model = model.to(device)
    criterion = criterion.to(device)


    return model, criterion

def set_model_both(opt):

    device = opt.device
    model1 = ResNet(name=opt.model,num_classes = opt.ncls)
    criterion1 = torch.nn.BCEWithLogitsLoss()
    model1 = model1.to(device)
    criterion1 = criterion1.to(device)

    model2 = MLP(n_inputs = 2, n_outputs = opt.ncls)
    criterion2 = torch.nn.BCEWithLogitsLoss()
    model2 = model2.to(device)
    criterion2 = criterion2.to(device)

    return model1, criterion1, model2, criterion2




def set_loader(opt):
    # construct data loader
    if opt.dataset == 'OLIVES' or opt.dataset == 'RECOVERY':
        mean = (.1706)
        std = (.2112)
    else:
        raise ValueError('dataset not supported: {}'.format(opt.dataset))

    normalize = transforms.Normalize(mean=mean, std=std)

    train_transform = transforms.Compose([
        transforms.RandomResizedCrop(size=224, scale=(0.2, 1.)),
        transforms.RandomHorizontalFlip(),

        transforms.RandomApply([
            transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)
        ], p=0.8),
        transforms.RandomGrayscale(p=0.2),
        transforms.ToTensor(),
        normalize,
    ])

    val_transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize,
    ])


    if opt.dataset =='OLIVES':
        data_path_train = opt.train_image_path
        data_path_test = opt.test_image_path
        train_dataset = OLIVES(csv_path_train,data_path_train,transforms = train_transform)
        unlabelled_train_dataset = RECOVERY(csv_path_unlabelled,data_path_train,transforms = val_transform)
        val_dataset = OLIVES(csv_path_valid,data_path_train,transforms = val_transform)
        test_dataset = RECOVERY(csv_path_test,data_path_test,transforms = val_transform)
    else:
        raise ValueError(opt.dataset)

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=opt.batch_size, shuffle=True,
        num_workers=opt.num_workers, pin_memory=True)
    
    val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=1, shuffle=False,
        num_workers=0, pin_memory=True,drop_last=False)
    
    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=1, shuffle=False,
        num_workers=0, pin_memory=True,drop_last=False)

    return train_loader, val_loader, test_loader




class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res


def adjust_learning_rate(args, optimizer, epoch):
    lr = args.learning_rate
    if args.cosine:
        eta_min = lr * (args.lr_decay_rate ** 3)
        lr = eta_min + (lr - eta_min) * (
                1 + math.cos(math.pi * epoch / args.epochs)) / 2
    else:
        steps = np.sum(epoch > np.asarray(args.lr_decay_epochs))
        if steps > 0:
            lr = lr * (args.lr_decay_rate ** steps)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


def warmup_learning_rate(args, epoch, batch_id, total_batches, optimizer):
    if args.warm and epoch <= args.warm_epochs:
        p = (batch_id + (epoch - 1) * total_batches) / \
            (args.warm_epochs * total_batches)
        lr = args.warmup_from + p * (args.warmup_to - args.warmup_from)

        for param_group in optimizer.param_groups:
            param_group['lr'] = lr


def set_optimizer(opt, model):

    optimizer = optim.SGD(model.parameters(),
                          lr=opt.learning_rate,
                          momentum=opt.momentum,
                          weight_decay=opt.weight_decay)


    return optimizer

def set_optimizer_both(opt, model1, model2):

    optimizer1 = optim.SGD(model1.parameters(),
                          lr=opt.learning_rate,
                          momentum=opt.momentum,
                          weight_decay=opt.weight_decay)

    optimizer2 = optim.Adam(model2.parameters(), lr=0.1, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=True)


    return optimizer1, optimizer2

def save_model(model, optimizer, opt, epoch, save_file):
    print('==> Saving...')
    state = {
        'opt': opt,
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'epoch': epoch,
    }
    torch.save(state, save_file)
    del state

In [14]:
def set_loader_clinical(opt):
    # construct data loader
    if opt.dataset == 'OLIVES' or opt.dataset == 'RECOVERY':
        mean = (.1706)
        std = (.2112)
    else:
        raise ValueError('dataset not supported: {}'.format(opt.dataset))

    normalize = transforms.Normalize(mean=mean, std=std)

    train_transform = transforms.Compose([
        transforms.RandomResizedCrop(size=224, scale=(0.8, 1.)),

        transforms.RandomApply([
            transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)
        ], p=0.8),
        transforms.RandomGrayscale(p=0.2),
        transforms.ToTensor(),
        normalize,
    ])

    val_transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize,
    ])


    if opt.dataset =='OLIVES':
        csv_path_train = opt.train_csv_path
        csv_path_test = opt.test_csv_path
        data_path_train = opt.train_image_path
        data_path_test = opt.test_image_path
        train_dataset = CLINICAL(clinical_path_train, data_path_train,transforms = train_transform)
        test_dataset = CLINICAL(clinical_path_test, data_path_test,transforms = val_transform)
        
    else:
        raise ValueError(opt.dataset)

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=opt.batch_size, shuffle=True,
        num_workers=opt.num_workers, pin_memory=True)
    
    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=1, shuffle=False,
        num_workers=0, pin_memory=True,drop_last=False)

    return train_loader, test_loader



In [15]:
# config.py

import argparse
import math
import os

def parse_option(string):
    parser = argparse.ArgumentParser('argument for training')

    parser.add_argument('--print_freq', type=int, default=10,
                        help='print frequency')
    parser.add_argument('--save_freq', type=int, default=50,
                        help='save frequency')
    parser.add_argument('--batch_size', type=int, default=128,
                        help='batch_size')
    parser.add_argument('--num_workers', type=int, default=8,
                        help='num of workers to use')
    parser.add_argument('--epochs', type=int, default=100,
                        help='number of training epochs')
    parser.add_argument('--device', type=str, default='cuda:0')
    # optimization
    parser.add_argument('--learning_rate', type=float, default=0.05,
                        help='learning rate')
    parser.add_argument('--patient_lambda', type=float, default=1,
                        help='learning rate')
    parser.add_argument('--cluster_lambda', type=float, default=1,
                        help='learning rate')
    parser.add_argument('--lr_decay_epochs', type=str, default='100',
                        help='where to decay lr, can be a list')
    parser.add_argument('--lr_decay_rate', type=float, default=0.1,
                        help='decay rate for learning rate')
    parser.add_argument('--weight_decay', type=float, default=1e-4,
                        help='weight decay')
    parser.add_argument('--momentum', type=float, default=0.9,
                        help='momentum')
    parser.add_argument('--train_csv_path', type=str, default='train data csv')
    parser.add_argument('--test_csv_path', type=str, default='test data csv')
    parser.add_argument('--train_image_path', type=str, default='train data csv')
    parser.add_argument('--test_image_path', type=str, default='test data csv')

    parser.add_argument('--parallel', type=int, default=1, help='data parallel')
    parser.add_argument('--ncls', type=int, default=6, help='Number of Classes')
    # model dataset
    parser.add_argument('--model', type=str, default='resnet50')
    parser.add_argument('--dataset', type=str, default='TREX_DME',
                        choices=[ 'OLIVES'], help='dataset')
    parser.add_argument('--mean', type=str, help='mean of dataset in path in form of str tuple')
    parser.add_argument('--std', type=str, help='std of dataset in path in form of str tuple')
    parser.add_argument('--data_folder', type=str, default=None, help='path to custom dataset')
    parser.add_argument('--size', type=int, default=128, help='parameter for RandomResizedCrop')

    # temperature
    parser.add_argument('--temp', type=float, default=0.07,
                        help='temperature for loss function')



    opt = parser.parse_args(string)

    # check if dataset is path that passed required arguments
    if opt.dataset == 'path':
        assert opt.data_folder is not None \
               and opt.mean is not None \
               and opt.std is not None

    # set the path according to the environment
    if opt.data_folder is None:
        opt.data_folder = './datasets/'
    opt.model_path = './save/{}_models'.format(opt.dataset)

    iterations = opt.lr_decay_epochs.split(',')
    opt.lr_decay_epochs = list([])
    for it in iterations:
        opt.lr_decay_epochs.append(int(it))

    opt.model_name = '{}_lr_{}_decay_{}_bsz_{}_temp_{}'. \
        format(opt.model, opt.learning_rate,
               opt.weight_decay, opt.batch_size, opt.temp)


    opt.save_folder = os.path.join(opt.model_path, opt.model_name)
    if not os.path.isdir(opt.save_folder):
        os.makedirs(opt.save_folder)

    return opt

In [60]:
def train_supervised_clinical(train_loader, model, criterion, optimizer, epoch, opt):
    """one epoch training"""
    model.train()


    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    device = opt.device
    end = time.time()

    for idx, (image, clinical) in enumerate(train_loader):
        data_time.update(time.time() - end)

        images = image.to(device)
        labels = clinical.to(torch.float32).to(device)
        bsz = labels.shape[0]

        # compute loss
        output = model(images)
        loss = criterion(output, labels)

        # update metric
        losses.update(loss.item(), bsz)
        #print(loss.item(), output.mean(axis = 0), labels.mean(axis = 0))
        
        # SGD
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # print info
        if (idx + 1) % opt.print_freq == 0:
            print('Train: [{0}][{1}/{2}]\t'.format(
                epoch, idx + 1, len(train_loader)))

            sys.stdout.flush()
    print("Training MSE Loss:", losses.avg)
    
    return losses.avg


In [17]:
def train_supervised(train_loader, model,criterion, optimizer, epoch, opt):
    """one epoch training"""
    model.train()


    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    device = opt.device
    end = time.time()
    correct_predictions = 0

    for idx, (image, bio_tensor) in enumerate(train_loader):
        data_time.update(time.time() - end)

        images = image.to(device)

        labels = bio_tensor.float()

        labels = labels.to(device)
        bsz = labels.shape[0]

        # compute loss
        output = model(images)
        loss = criterion(output, labels)
        
        # Calculate training accuracy
        predicted_labels = torch.round(torch.sigmoid(output)) 
        correct_predictions += (predicted_labels == labels).sum().item()

        # update metric
        losses.update(loss.item(), bsz)

        # SGD
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # print info
        if (idx + 1) % opt.print_freq == 0:
            print('Train: [{0}][{1}/{2}]\t'.format(
                epoch, idx + 1, len(train_loader)))

            sys.stdout.flush()

    total_values = len(train_loader.dataset) * 6
    training_accuracy = (correct_predictions / total_values) * 100.0
    print(f"Training Accuracy: {training_accuracy:.2f}%")
    
    return losses.avg


In [18]:
def train_supervised_hybrid(train_loader, val_loader, model, criterion, optimizer, epoch, opt):
    """one epoch training"""
    model.train()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    device = opt.device
    end = time.time()
    correct_predictions = 0

    for idx, (image, clinical, bio_tensor) in enumerate(train_loader):
        data_time.update(time.time() - end)

        images = image.to(torch.float32).to(device)
        clinical = clinical.to(torch.float32).to(device)

        labels = bio_tensor.to(torch.float32)

        labels = labels.to(device)
        bsz = labels.shape[0]

        # compute loss
        output = model(images, clinical)
        loss = criterion(output, labels)
        
        # Calculate training accuracy
        predicted_labels = torch.round(torch.sigmoid(output)) 
        correct_predictions += (predicted_labels == labels).sum().item()

        # update metric
        losses.update(loss.item(), bsz)

        # SGD
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # print info
        if (idx + 1) % opt.print_freq == 0:
            print('Train: [{0}][{1}/{2}]\t'.format(
                epoch, idx + 1, len(train_loader)))

            sys.stdout.flush()

    total_values = len(train_loader.dataset) * 6
    training_accuracy = (correct_predictions / total_values) * 100.0
    print(f"Training Accuracy: {training_accuracy:.2f}%")
    print(losses.avg)
    
    return losses.avg


In [19]:
def train_supervised_multimodal(train_loader, model1, model2, criterion1, criterion2, optimizer1, optimizer2, epoch, opt):
    """one epoch training"""
    model1.train()
    model2.train()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    device = opt.device
    end = time.time()
    correct_predictions = 0

    for idx, (image, clinical, bio_tensor) in enumerate(train_loader):
        data_time.update(time.time() - end)
        
        model1.zero_grad()
        optimizer1.zero_grad()
        model2.zero_grad()
        optimizer2.zero_grad()

        images = image.to(device)
        clinical = clinical.to(device)

        labels = bio_tensor.float()

        labels = labels.to(device)
        bsz = labels.shape[0]

        # compute loss
        output1 = model1(images)
        loss1 = criterion1(output1, labels)
        
        output2 = model2(clinical)
        loss2 = criterion2(output2, labels)
        
        # Calculate training accuracy
        pred1 = torch.round(torch.sigmoid(output1)) 
        pred2 = torch.round(torch.sigmoid(output2)) 
        correct_predictions += (pred1 == labels).sum().item()

        comp_criterion = nn.MSELoss()

        loss3 = comp_criterion(output1[pred2 == labels], output2[pred2 == labels])
        if torch.isnan(loss3):
            loss3 = comp_criterion(output1[pred1 == labels], output2[pred1 == labels])
        if torch.isnan(loss3):
            loss = loss1 + loss2
        else:
            loss = loss1 + loss2 + loss3
        # update metric
        losses.update(loss.item(), bsz)

        # SGD
        loss.backward()
        optimizer1.step()
        optimizer2.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # print info
        if (idx + 1) % opt.print_freq == 0:
            print('Train: [{0}][{1}/{2}]\t'.format(
                epoch, idx + 1, len(train_loader)))

            sys.stdout.flush()

    total_values = len(train_loader.dataset) * 6
    training_accuracy = (correct_predictions / total_values) * 100.0
    print(f"Training Accuracy: {training_accuracy:.2f}%")
    print(losses.avg)
    
    
    return losses.avg

In [20]:
def submission_generate(val_loader, model, opt):
    """validation"""
    model.eval()

    device = opt.device
    out_list = []
    with torch.no_grad():
        for idx, (image, clinical) in (enumerate(val_loader)):

            images = image.float().to(device)
            clinical = clinical.to(torch.float32).to(device)

            # forward
            output = model(images, clinical)
            output = torch.round(torch.sigmoid(output))
            out_list.append(output.squeeze().detach().cpu().numpy())


    out_submisison = np.array(out_list)
    np.save('output',out_submisison)


In [21]:
def validation_clinical(val_loader, model, criterion, opt):
    """validation"""
    model.eval()

    device = opt.device
    losses = AverageMeter()

    with torch.no_grad():
        for idx, (image, clinical) in (enumerate(val_loader)):

            images = image.float().to(device)
            labels = clinical.to(torch.float32).to(device)
            bsz = labels.shape[0]

            output = model(images)
            loss = criterion(output, labels)
            
            losses.update(loss.item(), bsz)
    print("Test MSE Loss:", losses.avg)
    return losses.avg


In [22]:
def sample_evaluation_acc(val_loader, model, opt):
    """validation"""
    model.eval()

    device = opt.device
    correct_count = 0
    total_count = 0

    with torch.no_grad():
        for idx, (image,clinical, bio_tensor) in (enumerate(val_loader)):

            images = image.float().to(device)
            clinical = clinical.to(torch.float32).to(device)
            labels = bio_tensor.float().to(device)

            labels = labels.float()

            output = model(images, clinical)
            output = torch.round(torch.sigmoid(output))
            
            correct_count += (labels == output).sum().item()
        
    total_count = len(val_loader.dataset) * 6
    val_acc = (correct_count / total_count) * 100
    print(val_acc, "%")
    return val_acc


In [23]:
def sample_evaluation(val_loader, model, opt):
    """validation"""
    model.eval()

    device = opt.device
    out_list = []
    label_list = []
    correct_count = 0
    total_count = 0

    with torch.no_grad():
        for idx, (image,clinical, bio_tensor) in (enumerate(val_loader)):

            images = image.float().to(device)
            clinical = clinical.to(torch.float32).to(device)
            labels = bio_tensor.float().to(device)

            labels = labels.float()

            label_list.append(labels.squeeze().detach().cpu().numpy())
            # forward
            output = model(images, clinical)
            output = torch.round(torch.sigmoid(output))
            out_list.append(output.squeeze().detach().cpu().numpy())
            
            correct_count += (labels == output).sum().item()
        
    total_count = len(val_loader.dataset) * 6
    print((correct_count / total_count) * 100, "%")

    label_array = np.array(label_list)
    out_array = np.array(out_list)
    f = f1_score(label_array,out_array,average='macro')
    print(f)

In [24]:
args = args = ['--batch_size', '64', '--model', "resnet50", '--dataset', 'OLIVES', '--epochs', '100', '--device', 'cuda:0', '--train_image_path', '/kaggle/input/olives-vip-cup-2023/2023 IEEE SPS Video and Image Processing (VIP) Cup - Ophthalmic Biomarker Detection/TRAIN/OLIVES', '--test_image_path', '/kaggle/input/olives-vip-cup-2023/2023 IEEE SPS Video and Image Processing (VIP) Cup - Ophthalmic Biomarker Detection/TEST/', '--test_csv_path', '/kaggle/input/olives-vip-cup-2023/2023 IEEE SPS Video and Image Processing (VIP) Cup - Ophthalmic Biomarker Detection/TEST/test_set_submission_template.csv', '--train_csv_path', '/kaggle/input/olives-training-labels/Training_Biomarker_Data.csv']
opt = parse_option(args)

In [56]:
# build data loader
opt.batch_size = 10
train_loader, test_loader = set_loader_clinical(opt)

/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [46]:
train_loader.dataset[0]

(tensor([[[-0.8078, -0.8078, -0.8078,  ..., -0.6778, -0.6221, -0.6407],
          [-0.8078, -0.8078, -0.8078,  ..., -0.6221, -0.4178, -0.5107],
          [-0.8078, -0.8078, -0.8078,  ..., -0.5664, -0.4178, -0.6778],
          ...,
          [-0.7706, -0.5849, -0.4735,  ..., -0.8078, -0.8078, -0.8078],
          [-0.7521, -0.7706, -0.7892,  ..., -0.8078, -0.8078, -0.8078],
          [-0.7892, -0.8078, -0.8078,  ..., -0.8078, -0.8078, -0.8078]]]),
 tensor([0.7300, 0.6405], dtype=torch.float64))

In [62]:
net = Encoder()
projection_head = ProjectionHead(512, 2)
clinical_model = nn.Sequential(net, projection_head)
criterion = nn.MSELoss()

clinical_model = clinical_model.to(opt.device)
criterion = criterion.to(opt.device)

optimizer = set_optimizer(opt, clinical_model)
optimizer = torch.optim.Adam(clinical_model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [71]:
epochs = 10
opt.print_freq = 10

with torch.autograd.set_detect_anomaly(True):
    for epoch in range(1, epochs + 1):
        train_supervised_clinical(train_loader, clinical_model, criterion, optimizer, epoch, opt)
        #loss = validation_clinical(test_loader, clinical_model, criterion, opt)
        scheduler.step()

Train: [1][10/100]	
Train: [1][20/100]	
Train: [1][30/100]	
Train: [1][40/100]	
Train: [1][50/100]	
Train: [1][60/100]	
Train: [1][70/100]	
Train: [1][80/100]	
Train: [1][90/100]	
Train: [1][100/100]	
Training MSE Loss: 0.002808182584121823
Train: [2][10/100]	
Train: [2][20/100]	
Train: [2][30/100]	
Train: [2][40/100]	
Train: [2][50/100]	
Train: [2][60/100]	
Train: [2][70/100]	
Train: [2][80/100]	
Train: [2][90/100]	
Train: [2][100/100]	
Training MSE Loss: 0.0025240870914421975
Train: [3][10/100]	
Train: [3][20/100]	
Train: [3][30/100]	
Train: [3][40/100]	
Train: [3][50/100]	
Train: [3][60/100]	
Train: [3][70/100]	
Train: [3][80/100]	
Train: [3][90/100]	
Train: [3][100/100]	
Training MSE Loss: 0.0028604165452998133
Train: [4][10/100]	
Train: [4][20/100]	
Train: [4][30/100]	
Train: [4][40/100]	
Train: [4][50/100]	
Train: [4][60/100]	
Train: [4][70/100]	
Train: [4][80/100]	
Train: [4][90/100]	
Train: [4][100/100]	
Training MSE Loss: 0.0028761536750243977
Train: [5][10/100]	
Train: [5][20

In [72]:
clinical_model.eval()
image, label = train_loader.dataset[13]
image = image.reshape((1, ) + image.shape).to(opt.device)
clinical_model(image), label

(tensor([[0.6921, 0.6226]], device='cuda:0', grad_fn=<AddmmBackward0>),
 tensor([0.7300, 0.6405], dtype=torch.float64))

In [40]:
clinical_model.train()
image,label = next(iter(train_loader))
image = image.to(opt.device)
clinical_model(image)

/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


tensor([[0.8049, 3.7109],
        [0.8049, 3.7109],
        [0.8049, 3.7109],
        [0.8049, 3.7109],
        [0.8049, 3.7109],
        [0.8049, 3.7109],
        [0.8049, 3.7109],
        [0.8049, 3.7109],
        [0.8049, 3.7109],
        [0.8049, 3.7109],
        [0.8049, 3.7109],
        [0.8049, 3.7109],
        [0.8049, 3.7109],
        [0.8049, 3.7109],
        [0.8049, 3.7109],
        [0.8049, 3.7109],
        [0.8049, 3.7109],
        [0.8049, 3.7109],
        [0.8049, 3.7109],
        [0.8049, 3.7109],
        [0.8049, 3.7109],
        [0.8065, 3.7140],
        [0.8049, 3.7109],
        [0.8049, 3.7109],
        [0.8049, 3.7109],
        [0.8049, 3.7109],
        [0.8049, 3.7109],
        [0.8049, 3.7109],
        [0.8049, 3.7109],
        [0.8049, 3.7109],
        [0.8049, 3.7109],
        [0.8049, 3.7109],
        [0.8049, 3.7109],
        [0.8049, 3.7109],
        [0.8049, 3.7109],
        [0.8049, 3.7109],
        [0.8049, 3.7109],
        [0.8049, 3.7109],
        [0.8

In [43]:
label.mean(axis = 0)

tensor([0.7958, 3.4642], dtype=torch.float64)

In [40]:
((label - output) ** 2).mean()

tensor(0.2867, device='cuda:0', grad_fn=<MeanBackward0>)

In [39]:
torch.cuda.empty_cache()

In [ ]:
# build data loader
train_loader, val_loader, test_loader = set_loader(opt)

In [ ]:
# build model and criterion
model, criterion = set_model_hybrid(opt)

In [ ]:
# build optimizer
optimizer = set_optimizer(opt, model)

In [ ]:
opt.learning_rate

In [ ]:
# training routine
best = 0
for epoch in range(1, 80 + 1):
    train_supervised_hybrid(train_loader, val_loader, model, criterion, optimizer, epoch, opt)
    val_acc = sample_evaluation_acc(val_loader, model, opt)
    if val_acc > best:
        best = val_acc
        save_model(model, optimizer, opt, opt.epochs, "/kaggle/working/last_model.pth")

In [ ]:
save_file = os.path.join(opt.save_folder, 'last2.pth')
save_model(model, optimizer, opt, opt.epochs, save_file)

In [ ]:
# Validation
sample_evaluation(val_loader, model, opt)

In [ ]:
submission_generate(test_loader, model, opt)

In [ ]:
output = np.load('/kaggle/working/output.npy')
submission = pd.read_csv("/kaggle/input/olives-vip-cup-2023/2023 IEEE SPS Video and Image Processing (VIP) Cup - Ophthalmic Biomarker Detection/TEST/test_set_submission_template.csv")
submission.iloc[:, 1:] = output
submission.to_csv("/kaggle/working/submission.csv", index = False)

In [ ]:
len(train_loader.dataset)

In [ ]:
image, clin, bio = val_loader.dataset[25]
image = image.reshape((1, ) + image.shape).to(opt.device)
clin = clin.reshape((1, ) + clin.shape).to(opt.device)
output = model(image, clin)
torch.round(torch.sigmoid(output)), bio, clin

In [ ]:
clin = torch.tensor([-0.5, 0.6]).reshape(clin.shape).to(opt.device)
output = model(image, clin)
torch.round(torch.sigmoid(output))

In [ ]:
output

In [ ]:
output

In [ ]:
clin

In [ ]:
import pandas as pd

df = pd.read_csv("/kaggle/input/olives-vip-cup-2023/2023 IEEE SPS Video and Image Processing (VIP) Cup - Ophthalmic Biomarker Detection/TRAIN/Training_Biomarker_Data.csv")
clinical_df = pd.read_excel("/kaggle/input/olives-vip-cup-2023/2023 IEEE SPS Video and Image Processing (VIP) Cup - Ophthalmic Biomarker Detection/TRAIN/Training_Unlabeled_Clinical_Data.xlsx")


In [ ]:
clinical_df.iloc[1].File_Path in df.iloc[:, 0]

In [ ]:
filtered_df = pd.DataFrame(columns = clinical_df.columns)
file_names = list(df.iloc[:, 0])

for i in range(len(clinical_df)):
    if clinical_df.iloc[i].File_Path in file_names:
        filtered_df.loc[len(filtered_df.index)] = clinical_df.loc[i]
        
filtered_df

In [ ]:
df.sort_values(by = df.columns[0])

In [ ]:
image, cln, bio = train_loader.dataset[0]
image.to(torch.float32)

In [67]:
df = pd.read_csv("/kaggle/input/olives-training-labels/Training_Unlabeled_Clinical_Data.csv")

df.BCVA.isnull()

0        False
1        False
2        False
3        False
4        False
         ...  
78131    False
78132    False
78133    False
78134    False
78135    False
Name: BCVA, Length: 78136, dtype: bool

In [48]:
for i in range(len(df)):
    if np.isnan(df.BCVA[i]) or np.isnan(df.CST[i]):
        print(i)

In [68]:
df.iloc[:10]

,File_Path,BCVA,CST,Eye_ID,Patient_ID
0,/TREX_DME/TREX DME/GILA/0234GOD/V1/OD/TREXW_00...,73,517,12,234
1,/TREX_DME/TREX DME/GILA/0234GOD/V1/OD/TREXW_00...,73,517,12,234
2,/TREX_DME/TREX DME/GILA/0234GOD/V1/OD/TREXW_00...,73,517,12,234
3,/TREX_DME/TREX DME/GILA/0234GOD/V1/OD/TREXW_00...,73,517,12,234
4,/TREX_DME/TREX DME/GILA/0234GOD/V1/OD/TREXW_00...,73,517,12,234
5,/TREX_DME/TREX DME/GILA/0234GOD/V1/OD/TREXW_00...,73,517,12,234
6,/TREX_DME/TREX DME/GILA/0234GOD/V1/OD/TREXW_00...,73,517,12,234
7,/TREX_DME/TREX DME/GILA/0234GOD/V1/OD/TREXW_00...,73,517,12,234
8,/TREX_DME/TREX DME/GILA/0234GOD/V1/OD/TREXW_00...,73,517,12,234
9,/TREX_DME/TREX DME/GILA/0234GOD/V1/OD/TREXW_00...,73,517,12,234


In [ ]:
clinical_df

In [ ]:
import re
regex = re.compile(r"RECOVERY/OCT/([0-9\-]+)/W([0-9]+)/")

In [ ]:
id_to_row = dict(zip(clinical_df['Patient ID'], clinical_df.index))
cols = ["Patient ID", "BCVA", "CST"]
for col in cols:
    df[col] = np.nan

for i in range(len(df)):
    patient_id, week = regex.findall(df.iloc[i, 0])[0]
    week = int(week)
    idx = id_to_row[patient_id]
    df.loc[i, cols[0]] = clinical_df.iloc[idx, 0]
    if week < 50:
        df.loc[i, cols[1]] = clinical_df.loc[idx, "Week 0 BCVA"]
        df.loc[i, cols[2]] = clinical_df.loc[idx, "Week 0 CST"]
    else:
        df.loc[i, cols[1]] = clinical_df.loc[idx, "Final Week BCVA"]
        df.loc[i, cols[2]] = clinical_df.loc[idx, "Final Week CST"]


In [ ]:
((df.BCVA - df.BCVA.mean())/df.BCVA.std()).unique()

In [ ]:
df.loc[i, clinical_df.columns] = clinical_df.iloc[idx]

In [ ]:
clinical_df.iloc[28, [3, 4]] = clinical_df.iloc[28, [1, 2]]

In [ ]:
clinical_df

In [ ]:
import numpy as np
arr = np.array([1,2,3,4,5,6,7])
test = np.array([1,2,3,1,2])
test[test == np.array([1,5,3,1,1])]

In [ ]:
bio_df = pd.read_csv(opt.test_csv_path)
clin_df = pd.read_excel(clinical_path_test)

df = prepare_clinical_data_test(opt.test_csv_path, clinical_path_test)

In [ ]:
df = df.drop(["B1", "B2", "B3", "B4", "B5", "B6"], axis = 1)

In [ ]:
df.to_csv("test_with_clinical.csv", index = False)